In [1]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.pardir))
import CONFIG
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn.functional as F
from sklearn import preprocessing

In [3]:
DATA_DIR = CONFIG.CFG.DATA.BASE
K_FOLDS = 5
BATCH_SIZE = 32
LEARNING_RATE = 3e-3
NUM_EPOCHS = 1000
ES_PATIENCE = 20
QUANTILES = (0.2, 0.5, 0.8)
SCALE_COLUMNS = ['Weeks', 'FVC', 'Percent', 'Age']
SEX_COLUMNS = ['Male', 'Female']
SMOKING_STATUS_COLUMNS = ['Currently smokes', 'Ex-smoker', 'Never smoked']
FV = SEX_COLUMNS + SMOKING_STATUS_COLUMNS + SCALE_COLUMNS
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [72]:
MIN_MAX_SCALER = preprocessing.MinMaxScaler()

In [73]:
train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
train_df.drop_duplicates(keep=False, inplace=True, subset=['Patient', 'Weeks'])
test_df = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))

In [74]:
train_df['Weeks'] = train_df['Weeks'] + 13
train_df[SCALE_COLUMNS] = MIN_MAX_SCALER.fit_transform(train_df[SCALE_COLUMNS])

In [76]:
train_df.sample(5)

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
388,ID00109637202210454292264,0.434783,0.236001,0.216264,0.615385,Male,Ex-smoker
207,ID00061637202188184085559,0.471014,0.551866,0.626009,0.487179,Male,Ex-smoker
981,ID00264637202270643353440,0.130435,0.199390,0.690202,0.692308,Female,Never smoked
952,ID00248637202266698862378,0.231884,0.099964,0.464174,0.564103,Female,Never smoked
622,ID00165637202237320314458,0.789855,0.263640,0.240543,0.128205,Male,Ex-smoker


In [59]:
# REMOVE THE ONES FROM THE TRAIN_DF THAT ARE PRESENT IN TEST_DF AS WELL

In [77]:
# convert categoricals into dummies
train_df['Sex'] = pd.Categorical(train_df['Sex'], categories=SEX_COLUMNS)
train_df['SmokingStatus'] = pd.Categorical(train_df['SmokingStatus'], categories=SMOKING_STATUS_COLUMNS)
train_df = train_df.join(pd.get_dummies(train_df['Sex']))
train_df = train_df.join(pd.get_dummies(train_df['SmokingStatus']))

In [78]:
train_df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,Male,Female,Currently smokes,Ex-smoker,Never smoked
0,ID00007637202177411956430,0.007246,0.267050,0.236393,0.769231,Male,Ex-smoker,1,0,0,1,0
1,ID00007637202177411956430,0.072464,0.248923,0.215941,0.769231,Male,Ex-smoker,1,0,0,1,0
2,ID00007637202177411956430,0.086957,0.221464,0.184960,0.769231,Male,Ex-smoker,1,0,0,1,0
3,ID00007637202177411956430,0.101449,0.236360,0.201767,0.769231,Male,Ex-smoker,1,0,0,1,0
4,ID00007637202177411956430,0.115942,0.222900,0.186580,0.769231,Male,Ex-smoker,1,0,0,1,0


In [89]:
sub_df = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))
sub_df['Patient'] = sub_df['Patient_Week'].apply(lambda x: x.split('_')[0])
sub_df['Weeks'] = sub_df['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
sub_df.head()

,Patient_Week,FVC,Confidence,Patient,Weeks
0,ID00419637202311204720264_-12,2000,100,ID00419637202311204720264,-12
1,ID00421637202311550012437_-12,2000,100,ID00421637202311550012437,-12
2,ID00422637202311677017371_-12,2000,100,ID00422637202311677017371,-12
3,ID00423637202312137826377_-12,2000,100,ID00423637202312137826377,-12
4,ID00426637202313170790466_-12,2000,100,ID00426637202313170790466,-12


In [90]:
sub_df = sub_df.drop('FVC', axis=1).merge(test_df.drop('Weeks', axis=1), on='Patient')
sub_df['Weeks'] = sub_df['Weeks'] + 13

In [91]:
# have to make it categorical coz sub's sex column has males only
sub_df['Sex'] = pd.Categorical(sub_df['Sex'], categories=SEX_COLUMNS)
sub_df['SmokingStatus'] = pd.Categorical(sub_df['SmokingStatus'], categories=SMOKING_STATUS_COLUMNS)

In [92]:
sub_df = sub_df.join(pd.get_dummies(sub_df['Sex']))
sub_df = sub_df.join(pd.get_dummies(sub_df['SmokingStatus']))

In [93]:
sub_df.head()

,Patient_Week,Confidence,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,Male,Female,Currently smokes,Ex-smoker,Never smoked
0,ID00419637202311204720264_-12,100,ID00419637202311204720264,1,3020,70.186855,73,Male,Ex-smoker,1,0,0,1,0
1,ID00419637202311204720264_-11,100,ID00419637202311204720264,2,3020,70.186855,73,Male,Ex-smoker,1,0,0,1,0
2,ID00419637202311204720264_-10,100,ID00419637202311204720264,3,3020,70.186855,73,Male,Ex-smoker,1,0,0,1,0
3,ID00419637202311204720264_-9,100,ID00419637202311204720264,4,3020,70.186855,73,Male,Ex-smoker,1,0,0,1,0
4,ID00419637202311204720264_-8,100,ID00419637202311204720264,5,3020,70.186855,73,Male,Ex-smoker,1,0,0,1,0


In [96]:
sub_df[SCALE_COLUMNS] = MIN_MAX_SCALER.transform(sub_df[SCALE_COLUMNS])
sub_df.head()

,Patient_Week,Confidence,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,Male,Female,Currently smokes,Ex-smoker,Never smoked
0,ID00419637202311204720264_-12,100,ID00419637202311204720264,-0.050725,0.393575,0.332421,0.615385,Male,Ex-smoker,1,0,0,1,0
1,ID00419637202311204720264_-11,100,ID00419637202311204720264,-0.043478,0.393575,0.332421,0.615385,Male,Ex-smoker,1,0,0,1,0
2,ID00419637202311204720264_-10,100,ID00419637202311204720264,-0.036232,0.393575,0.332421,0.615385,Male,Ex-smoker,1,0,0,1,0
3,ID00419637202311204720264_-9,100,ID00419637202311204720264,-0.028986,0.393575,0.332421,0.615385,Male,Ex-smoker,1,0,0,1,0
4,ID00419637202311204720264_-8,100,ID00419637202311204720264,-0.021739,0.393575,0.332421,0.615385,Male,Ex-smoker,1,0,0,1,0


In [31]:
class PulmonaryDataset(Dataset):
    def __init__(self, df, FV, test=False):
        self.df = df
        self.test = test
        self.FV = FV

    def __getitem__(self, idx):
        return {
            'features': torch.tensor(self.df[self.FV].iloc[idx].values),
            'target': torch.tensor(self.df['FVC'].iloc[idx])
        }

    def __len__(self):
        return len(self.df)

In [166]:
class PulmonaryModel(nn.Module):
    def __init__(self, in_features=9, out_quantiles=3):
        super(PulmonaryModel, self).__init__()
        self.fc1 = nn.Linear(in_features, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, out_quantiles)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [110]:
def quantile_loss(preds, target, quantiles):
    assert not target.requires_grad
    assert preds.size(0) == target.size(0)
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i]
        losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(1))
    loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
    return loss

In [167]:
train_dataset = PulmonaryDataset(train_df, FV)
test_dataset = PulmonaryDataset(sub_df, FV)

train_data_loader = DataLoader(
    train_dataset,
    batch_size=10,
    drop_last=False,
    num_workers=2
)
test_data_loader = DataLoader(
    test_dataset,
    batch_size=10,
    drop_last=False,
    num_workers=2
)

In [132]:
model = PulmonaryModel(len(FV))
model.to(DEVICE)

PulmonaryModel(
  (fc1): Linear(in_features=9, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
)

In [133]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

In [134]:
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for i, data in enumerate(train_data_loader):
        features = data['features']
        targets = data['target']

        features = features.to(DEVICE).float()
        targets = targets.to(DEVICE).float()

        model.zero_grad()
        out = model(features)
        loss = quantile_loss(out, targets, QUANTILES)
        total_loss += loss
        loss.backward()
        optimizer.step()
        # scheduler.step()
    print(f"Epoch {epoch}, Loss {total_loss/len(train_data_loader)}")

Epoch 0, Loss 0.4136742949485779
Epoch 1, Loss 0.348447322845459
Epoch 2, Loss 0.28966638445854187
Epoch 3, Loss 0.23800082504749298
Epoch 4, Loss 0.1994795948266983
Epoch 5, Loss 0.17243745923042297
Epoch 6, Loss 0.1541115790605545
Epoch 7, Loss 0.14094488322734833
Epoch 8, Loss 0.13122762739658356
Epoch 9, Loss 0.12397663295269012
Epoch 10, Loss 0.11859326809644699
Epoch 11, Loss 0.11450060456991196
Epoch 12, Loss 0.1110667809844017
Epoch 13, Loss 0.10806529223918915
Epoch 14, Loss 0.10526511818170547
Epoch 15, Loss 0.10284237563610077
Epoch 16, Loss 0.10071488469839096
Epoch 17, Loss 0.09873451292514801
Epoch 18, Loss 0.09690078347921371
Epoch 19, Loss 0.0951591208577156
Epoch 20, Loss 0.0934789851307869
Epoch 21, Loss 0.09188125282526016
Epoch 22, Loss 0.09033752977848053
Epoch 23, Loss 0.08883313089609146
Epoch 24, Loss 0.08735959231853485
Epoch 25, Loss 0.0858839675784111
Epoch 26, Loss 0.08438540995121002
Epoch 27, Loss 0.08284483850002289
Epoch 28, Loss 0.08128354698419571
Epoc

In [181]:
preds = []
with torch.no_grad():
    for j, test_data in enumerate(test_data_loader):
        features = test_data['features']
        targets = test_data['target']

        features = features.to(DEVICE).float()
        targets = targets.to(DEVICE).float()

        out = model(features)
        for ou in out.cpu().numpy().tolist():
            preds.append(ou)

In [186]:
FVC_MIN = MIN_MAX_SCALER.data_min_[1]
FVC_MAX = MIN_MAX_SCALER.data_max_[1]

In [191]:
inv_preds = []
for a in preds:
    b = []
    for val in a:
        z = (FVC_MAX - FVC_MIN) * val + FVC_MIN
        b.append(z)
    inv_preds.append(b)

In [192]:
inv_preds

[[3017.6381200551987, 3012.2131536006927, 3015.8994871377945],
 [3018.4079674482346, 3012.7075098752975, 3017.2370886802673],
 [3019.1774827241898, 3013.2018661499023, 3018.5745241642],
 [3019.947496175766, 3013.696222424507, 3019.9121257066727],
 [3020.236936211586, 3013.964905142784, 3020.471909046173],
 [3020.1768230199814, 3014.4652395248413, 3020.4523141384125],
 [3020.1167098283768, 3014.9660720825195, 3020.432719230652],
 [3020.056596636772, 3015.4667385816574, 3020.4131243228912],
 [3019.9964834451675, 3015.967239022255, 3020.3935294151306],
 [3019.9367023706436, 3016.468071579933, 3020.37393450737],
 [3019.876589179039, 3016.9689041376114, 3020.3543395996094],
 [3019.8827333450317, 3016.9873366355896, 3020.341719150543],
 [3019.8902059793472, 3017.000123143196, 3020.329596877098],
 [3019.8975125551224, 3017.0134078264236, 3020.317474603653],
 [3019.904653072357, 3017.02602827549, 3020.305352330208],
 [3019.9121257066727, 3017.0389808416367, 3020.2930639982224],
 [3019.91910016